In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from matplotlib.colors import LinearSegmentedColormap
from statsmodels.stats.multitest import multipletests



# Header

## sub header

plain text here with some ```python``` commands

In [ ]:

# Define custom origin order
custom_origin_order = [
    "Os_LEA_or6", "syn_LEA_1", "Asyn_isoform_5", "A0A4D6M434", "A0A1U8F7A1",
    "At2g46140_tile7", "TgL_8x", "A0A438HAK2", "Os_LEA_or4",
    "CAHS_68135_tile4", "Os_LEA_or3__Kappa_up_2_"
]


In [ ]:


# === Step 1: Load Dataset ===
round_df = pd.read_csv('sequencing_data_all.csv')
round_df['Sequence_ID'] = round_df['Sequence_ID'].astype(str).str.strip()


In [ ]:

# === Step 2: Define features ===
nardini_cols = [col for col in round_df.columns if col.startswith('nardini_') and (col.endswith('_zscore') or col.endswith('_raw'))]

other_features = [
    'cider_kappa', 'cider_omega', 'cider_delta', 'cider_length', 
    'cider_mean_net_charge', 'cider_fraction_neutral',
    'sparrow_SCD', 'sparrow_SHD', 'sparrow_complexity',
    'sparrow_scaled_rg', 'sparrow_scaled_re', 
    'sparrow_prefactor', 'sparrow_scaling_exponent', 'sparrow_asphericity'
]

new_feature_cols = [
    'avg_helix_prob', 'avg_beta_prob', 'avg_coil_prob',
    'avg_mito_targeting', 'avg_nes', 'avg_nis'
]

all_features = nardini_cols + other_features + new_feature_cols
chem_cols = [col for col in all_features if col in round_df.columns]

print(f"Found {len(nardini_cols)} nardini columns")
print(f"Total features to analyze: {len(chem_cols)}")

df1 = round_df


In [ ]:

# === Step 5: Spearman r Filtering ===
results = []
n_iter = 1000
min_n = 6
rng = np.random.default_rng(42)

for base in custom_origin_order:
    base_df = df1[df1['base_origin'] == base].copy()
    original_id = f"original_sequence_-_{base}"
    original_row = df1[df1['Sequence_ID'] == original_id]

    if original_row.empty:
        print(f"⚠️  Original sequence not found for base: {base}")
        continue

    for feat in chem_cols:
        if feat not in df1.columns or pd.isna(original_row[feat].values[0]):
            continue

        if not pd.api.types.is_numeric_dtype(df1[feat]):
            print(f"Skipping non-numeric feature: {feat}")
            continue

        original_value = original_row[feat].values[0]
        lower_bound = original_value * 0.93
        upper_bound = original_value * 1.07

        filtered_df = base_df[
            ((base_df[feat] < lower_bound) | (base_df[feat] > upper_bound)) |
            (base_df['Sequence_ID'] == original_id)
        ].dropna(subset=[feat, 'log2FoldChange', 'lfcSE'])

        if len(filtered_df) < min_n:
            print(f"⚠️  Skipping {base} - {feat}: n={len(filtered_df)} < {min_n}")
            continue

        r_vals = []
        slope_vals = []
        for _ in range(n_iter):
            sample = filtered_df.sample(frac=1, replace=True, random_state=rng.integers(1e9))
            if sample[feat].nunique() > 1 and sample['log2FoldChange'].nunique() > 1:
                r = sample[feat].corr(sample['log2FoldChange'], method='spearman')
                if pd.notnull(r):
                    r_vals.append(r)
                slope, _, _, _, _ = stats.linregress(sample[feat], sample['log2FoldChange'])
                if pd.notnull(slope):
                    slope_vals.append(slope)

        if r_vals:
            r_full, p_full = stats.spearmanr(filtered_df[feat], filtered_df['log2FoldChange'])
            r_array = np.array(r_vals)
            bootstrap_p = 2 * min(np.mean(r_array > 0), np.mean(r_array < 0))
            bootstrap_p = max(bootstrap_p, 1/n_iter)
            
            results.append({
                'base_origin': base,
                'feature': feat,
                'r_mean': np.mean(r_vals),
                'r_std': np.std(r_vals),
                'slope_mean': np.mean(slope_vals) if slope_vals else np.nan,
                'slope_std': np.std(slope_vals) if slope_vals else np.nan,
                'p_value': p_full,
                'bootstrap_p': bootstrap_p,
                'n_samples': len(filtered_df),
                'mean_lfcSE': filtered_df['lfcSE'].mean()
            })


In [ ]:

# === Step 6: Apply FDR correction and save results ===
results_df = pd.DataFrame(results)

_, pvals_adj, _, _ = multipletests(results_df['p_value'], method='fdr_bh')
results_df['p_adj'] = pvals_adj
results_df['neg_log10_padj'] = -np.log10(results_df['p_adj'].clip(lower=1e-300))

_, bootstrap_pvals_adj, _, _ = multipletests(results_df['bootstrap_p'], method='fdr_bh')
results_df['bootstrap_p_adj'] = bootstrap_pvals_adj
results_df['neg_log10_bootstrap_padj'] = -np.log10(results_df['bootstrap_p_adj'].clip(lower=1e-300))

results_df.to_csv('spearman_2zn_results.csv', index=False)

print(f"\n✅ Total correlations calculated: {len(results_df)}")
print(f"Significant (p_adj < 0.05): {(results_df['p_adj'] < 0.05).sum()}")
print(f"Sample size statistics:")
print(results_df['n_samples'].describe())


In [ ]:

# === Step 7: Create clustered heatmap ===
heatmap_df = results_df.pivot(index='feature', columns='base_origin', values='r_mean')
heatmap_df = heatmap_df[custom_origin_order]
heatmap_df_clean = heatmap_df.dropna(how='all')
heatmap_df_transposed = heatmap_df_clean.T
heatmap_df_for_clustering = heatmap_df_transposed.fillna(0)

fig_width = len(heatmap_df_clean)*0.6 + 6
fig_height = len(custom_origin_order)*0.7 + 4

g = sns.clustermap(
    heatmap_df_for_clustering,
    cmap='coolwarm',
    center=0, vmin=-1, vmax=1,
    figsize=(fig_width, fig_height),
    linewidths=0.5,
    cbar_kws={'label': 'Mean Spearman r'},
    annot=True, fmt=".2f",
    method='average', metric='euclidean',
    row_cluster=True, col_cluster=True,
    dendrogram_ratio=0.15,
    cbar_pos=(0.92, 0.3, 0.03, 0.4)
)

g.fig.suptitle(f'{round_name}: Spearman r (Filtered ±7%, n≥{min_n})', fontsize=14, y=1.02)
g.ax_heatmap.set_xlabel('Feature', fontsize=12)
g.ax_heatmap.yaxis.set_ticks_position('right')
g.ax_heatmap.yaxis.set_label_position('right')
g.ax_heatmap.set_ylabel('base_origin', fontsize=12)

clustered_row_order = g.dendrogram_row.reordered_ind
clustered_col_order = g.dendrogram_col.reordered_ind
row_labels = heatmap_df_for_clustering.index[clustered_row_order]
col_labels = heatmap_df_for_clustering.columns[clustered_col_order]

plt.savefig('spearman_2zn_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

# === Step 7b: Slope/Amplitude Heatmap ===
print("\n📊 Creating slope heatmap...")
magenta_white_green = LinearSegmentedColormap.from_list('magenta_white_green', ['magenta', 'white', 'green'])

slope_heatmap = results_df.pivot(index='feature', columns='base_origin', values='slope_mean')
slope_heatmap_T = slope_heatmap.T.fillna(0)
slope_heatmap_ordered = slope_heatmap_T.reindex(index=row_labels, columns=col_labels)

fig, ax = plt.subplots(figsize=(fig_width, fig_height))
sns.heatmap(slope_heatmap_ordered, cmap=magenta_white_green, center=0, vmin=-3, vmax=3,
            linewidths=0.5, cbar_kws={'label': 'Slope (Amplitude)'}, annot=True, fmt=".1f", ax=ax)
ax.set_title(f'{round_name}: Slope/Amplitude (Filtered ±7%, n≥{min_n})', fontsize=14)
ax.set_xlabel('Feature', fontsize=12)
ax.yaxis.set_ticks_position('right')
ax.yaxis.set_label_position('right')
ax.set_ylabel('base_origin', fontsize=12)
plt.tight_layout()
plt.savefig('slope_amplitude_2zn_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

# === Step 7c: Analytic P-value Heatmap ===
print("\n📊 Creating analytic p-value heatmap...")
white_to_purple = LinearSegmentedColormap.from_list('white_purple', ['white', 'mediumpurple', 'darkviolet', 'indigo'])
sig_threshold = -np.log10(0.05)

pval_heatmap = results_df.pivot(index='feature', columns='base_origin', values='neg_log10_padj')
pval_heatmap_T = pval_heatmap.T.fillna(0)
pval_heatmap_ordered = pval_heatmap_T.reindex(index=row_labels, columns=col_labels)

fig, ax = plt.subplots(figsize=(fig_width, fig_height))
sns.heatmap(pval_heatmap_ordered, cmap=white_to_purple, vmin=0, vmax=max(pval_heatmap_ordered.values.max(), 5),
            linewidths=0.5, cbar_kws={'label': '-log10(adj p-value)'}, annot=True, fmt=".1f", ax=ax)
ax.set_title(f'{round_name}: Analytic Confidence -log10(FDR adj p)\n(values > {sig_threshold:.1f} are significant)', fontsize=14)
ax.set_xlabel('Feature', fontsize=12)
ax.yaxis.set_ticks_position('right')
ax.yaxis.set_label_position('right')
ax.set_ylabel('base_origin', fontsize=12)
plt.tight_layout()
plt.savefig('confidence_pvalue_2zn_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

# === Step 7d: r_std Heatmap (Bootstrap Uncertainty) ===
print("\n📊 Creating r_std (bootstrap uncertainty) heatmap...")
white_to_red = LinearSegmentedColormap.from_list('white_red', ['white', 'lightsalmon', 'red', 'darkred'])

rstd_heatmap = results_df.pivot(index='feature', columns='base_origin', values='r_std')
rstd_heatmap_T = rstd_heatmap.T.fillna(0)
rstd_heatmap_ordered = rstd_heatmap_T.reindex(index=row_labels, columns=col_labels)

fig, ax = plt.subplots(figsize=(fig_width, fig_height))
sns.heatmap(rstd_heatmap_ordered, cmap=white_to_red, vmin=0, vmax=0.3,
            linewidths=0.5, cbar_kws={'label': 'r_std (Bootstrap SD)'}, annot=True, fmt=".2f", ax=ax)
ax.set_title(f'{round_name}: Bootstrap Uncertainty (r_std)\n(lower = more stable)', fontsize=14)
ax.set_xlabel('Feature', fontsize=12)
ax.yaxis.set_ticks_position('right')
ax.yaxis.set_label_position('right')
ax.set_ylabel('base_origin', fontsize=12)
plt.tight_layout()
plt.savefig('rstd_uncertainty_2zn_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

# === Step 7e: Bootstrap Confidence Heatmap ===
print("\n📊 Creating bootstrap confidence heatmap...")
white_to_teal = LinearSegmentedColormap.from_list('white_teal', ['white', 'lightseagreen', 'teal', 'darkslategray'])

bootstrap_pval_heatmap = results_df.pivot(index='feature', columns='base_origin', values='neg_log10_bootstrap_padj')
bootstrap_pval_heatmap_T = bootstrap_pval_heatmap.T.fillna(0)
bootstrap_pval_heatmap_ordered = bootstrap_pval_heatmap_T.reindex(index=row_labels, columns=col_labels)

fig, ax = plt.subplots(figsize=(fig_width, fig_height))
sns.heatmap(bootstrap_pval_heatmap_ordered, cmap=white_to_teal, vmin=0, vmax=max(bootstrap_pval_heatmap_ordered.values.max(), 5),
            linewidths=0.5, cbar_kws={'label': '-log10(adj bootstrap p)'}, annot=True, fmt=".1f", ax=ax)
ax.set_title(f'{round_name}: Bootstrap Confidence -log10(FDR adj p)\n(values > {sig_threshold:.1f} are significant)', fontsize=14)
ax.set_xlabel('Feature', fontsize=12)
ax.yaxis.set_ticks_position('right')
ax.yaxis.set_label_position('right')
ax.set_ylabel('base_origin', fontsize=12)
plt.tight_layout()
plt.savefig('bootstrap_confidence_2zn_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Generated 5 heatmaps: correlation, slope, analytic p-value, r_std, and bootstrap p-value")

# === Step 8: Plot significant scatterplots (|r| >= 0.6) ===
print(f"\n📈 Plotting correlations with |r| >= 0.6...")
plot_count = 0

for _, row in results_df.iterrows():
    if abs(row['r_mean']) >= 0.6:
        base = row['base_origin']
        feat = row['feature']
        original_id = f"original_sequence_-_{base}"
        original_row = df1[df1['Sequence_ID'] == original_id]

        if original_row.empty or pd.isna(original_row[feat].values[0]):
            continue

        original_value = original_row[feat].values[0]
        lower_bound = original_value * 0.93
        upper_bound = original_value * 1.07

        plot_df = df1[
            (df1['base_origin'] == base) &
            ((df1['Sequence_ID'] == original_id) | 
             ((df1[feat] < lower_bound) | (df1[feat] > upper_bound)))
        ].dropna(subset=[feat, 'log2FoldChange', 'lfcSE'])

        if plot_df.shape[0] < min_n:
            continue

        plt.figure(figsize=(6, 4))
        plt.errorbar(plot_df[feat], plot_df['log2FoldChange'], yerr=plot_df['lfcSE'], 
                     fmt='o', alpha=0.6, ecolor='gray', elinewidth=1, capsize=2, label='Variants')

        if not original_row.empty:
            wt_se = original_row['lfcSE'].values[0] if pd.notna(original_row['lfcSE'].values[0]) else 0
            plt.errorbar(original_row[feat], original_row['log2FoldChange'], yerr=wt_se, 
                         fmt='o', color='red', markersize=8, label='WT')

        sns.regplot(x=feat, y='log2FoldChange', data=plot_df, scatter=False, ci=95, 
                    color='blue', line_kws={'label': 'Fit'})

        plt.xlabel(feat)
        plt.ylabel('log2FoldChange')
        plt.title(f'{base}: {feat} vs log2FoldChange\n(r={row["r_mean"]:.2f}, n={row["n_samples"]})')
        plt.legend()
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.savefig(f'scatter_{base}_{feat}.png', dpi=150, bbox_inches='tight')
        plt.show()
        plot_count += 1

print(f"\n✅ Generated {plot_count} scatter plots for correlations with |r| >= 0.6")